# 욜로(YOLO)를 이용한 객체 인식

In [ ]:
# 내 구글 드라이브에 연동

from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# 필요한 패키지와 모듈을 불러온다.

import cv2
import numpy as np
import time
import io
import base64
from IPython.display import HTML

In [ ]:
# OpenCV 버전 확인
print("OpenCV Version !!! ", cv2.__version__)

OpenCV Version !!!  4.7.0


In [ ]:
# Detection 하기 전에 원본 비디오를 Display 한다.

video = io.open('gdrive/My Drive/CV/Object Detection_YOLO/video/street.mp4', 'r+b').read()
encoded = base64.b64encode(video)
HTML(data='''<video width="30%" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4"/>
             </video>'''.format(encoded.decode('ascii')))

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
#  비디오에서 사물 검출 및 검출결과를 화면에 출력하는 함수 정의

def detectAndDisplay(frame):
    start_time = time.time()
    img = cv2.resize(frame, None, fx=0.9, fy=0.9)
    height, width, channels = img.shape

    # YOLOv3의 Detecting model 3가지(320×320, 416×416, 608×608)
    # 영상의 전처리
    blob = cv2.dnn.blobFromImage(img, 0.00392, (416, 416), (0, 0, 0), True, crop=False)

    net.setInput(blob)                   # 네트워크에 전처리된 영상 입력 설정
    outs = net.forward(output_layers)    # 네트워크 순방향 추론

    # Showing informations on the screen
    class_ids = []      # detection 한 Class id를 저장하는 배열 정의
    confidences = []    # detection 한 Class 의 신뢰도(확률)를 저장하는 배열 정의
    boxes = []          # detection 한 boxing 정보를 저장하는 배열 정의

    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)      # detection 한 Class id
            confidence = scores[class_id]     # detection 한 Class 의 신뢰도(확률)
            if confidence > min_confidence:
                # Object detected
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)

                # Rectangle coordinates
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)

                boxes.append([x, y, w, h])             # boxing 정보를 boxes 배열에 저장
                confidences.append(float(confidence))  # 신뢰도(확률)을 confidences 배열에 저장
                class_ids.append(class_id)             # Class id 를 class_ids 배열에 저장

    # apply non-max suppression
    # 박스안에 박스(노이즈)를 하나로 만들어 준다.
    indexes = cv2.dnn.NMSBoxes(boxes, confidences, min_confidence, nms_threshold)

    font = cv2.FONT_HERSHEY_PLAIN
    for i in range(len(boxes)):
        if i in indexes:    # 노이즈가 제거된 박스만 표시해 준다.
            x, y, w, h = boxes[i]

            # Class 이름, 신뢰도(확률) 표시
            label = "{}: {:.2f}".format(classes[class_ids[i]], confidences[i]*100)
            print(i, label)
            color = colors[i]
            cv2.rectangle(img, (x, y), (x + w, y + h), color, 1)
            cv2.rectangle(img, (x, y - 20), (x + w, y), color, -1)
            cv2.putText(img, label, (x + 5, y - 5), font, 1, (255, 255, 255), 1)
    process_time = time.time() - start_time
    global elapsed_time
    elapsed_time += process_time   # 총 경과시간 누적
    print("=== A frame took {:.3f} seconds".format(process_time))

    # video 를 disk 에 output 하기 위해 writer 를 초기화한다.
    global writer
    if writer is None and output_name is not None:
        fourcc = cv2.VideoWriter_fourcc(*"MJPG")
        writer = cv2.VideoWriter(output_name, fourcc, 30,
                (img.shape[1], img.shape[0]), True)

    # disk 에 frame 을 write 합니다.
    if writer is not None:
        writer.write(img)

In [ ]:
# detection 할 Object(Class) list 배열을 정의
classes = []

# 80개의 Object(class)를 구분할 수 있는 Object의 이름을 classes 배열에 넣어준다.
with open("gdrive/My Drive/CV/Object Detection_YOLO/config/coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]

print("classes len :", len(classes))    # classes len : 80
print("classes :", classes)

classes len : 80
classes : ['person', 'bicycle', 'car', 'motorbike', 'aeroplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'sofa', 'pottedplant', 'bed', 'diningtable', 'toilet', 'tvmonitor', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush']


In [ ]:
# 80개 부류를 구분하기 위해 난수를 이용해 서로 다른 색깔을 설정
# - Object 마다 컬러를 하나씩 다르게 지정
colors = np.random.uniform(0, 255, size=(len(classes), 3))

print("colors len :", len(colors))     # colors len : 80
print("colors shape :", colors.shape)  # colors shape : (80, 3)
print(colors[0:5])

colors len : 80
colors shape : (80, 3)
[[ 60.52578184  22.29611611  99.91424577]
 [ 25.21864294 230.33244361 183.30394192]
 [ 80.0520086  192.57667211 114.77651395]
 [ 37.96068188  94.0874041  176.37253073]
 [147.71110792 171.6720873  158.73406199]]


In [ ]:
# 하이퍼파라미터 설정

min_confidence = 0.5                            # detection 으로 인정할 최소 확률(신뢰도) 지정 : 50%

# 비최대 억제(Non-Maximum Suppression, NMS) 알고리즘을 적용해 주위 바운딩 박스에 비해 최대를 유지한 것 인덱스 확인한다.
#  - 박스안에 박스(노이즈)를 하나로 만들어 준다.

# 이 값이 클수록 box가 많이 사라진다(즉, 조금낭 겹쳐도 NMS로 둘 중 하나를 삭제하기 때문)
nms_threshold = 0.4

# YOLOv3

In [ ]:
# Load YOLOv3
# Darknet 프레임워크로 YOLO 네트워크 연결
net = cv2.dnn.readNet("gdrive/My Drive/CV/Object Detection_YOLO/config/yolov3.weights",
                      "gdrive/My Drive/CV/Object Detection_YOLO/config/yolov3.cfg")

In [ ]:
# YOLO 인공신경망 계층 이름 출력
layer_names = net.getLayerNames()
print("layer_name len :", len(layer_names))   # layer_name len : 254
print("layer_name:", layer_names)             # layer_name: ('conv_0', 'bn_0', 'leaky_1', 'conv_1', 'bn_1', 'leaky_2',

layer_name len : 254
layer_name: ('conv_0', 'bn_0', 'leaky_1', 'conv_1', 'bn_1', 'leaky_2', 'conv_2', 'bn_2', 'leaky_3', 'conv_3', 'bn_3', 'leaky_4', 'shortcut_4', 'conv_5', 'bn_5', 'leaky_6', 'conv_6', 'bn_6', 'leaky_7', 'conv_7', 'bn_7', 'leaky_8', 'shortcut_8', 'conv_9', 'bn_9', 'leaky_10', 'conv_10', 'bn_10', 'leaky_11', 'shortcut_11', 'conv_12', 'bn_12', 'leaky_13', 'conv_13', 'bn_13', 'leaky_14', 'conv_14', 'bn_14', 'leaky_15', 'shortcut_15', 'conv_16', 'bn_16', 'leaky_17', 'conv_17', 'bn_17', 'leaky_18', 'shortcut_18', 'conv_19', 'bn_19', 'leaky_20', 'conv_20', 'bn_20', 'leaky_21', 'shortcut_21', 'conv_22', 'bn_22', 'leaky_23', 'conv_23', 'bn_23', 'leaky_24', 'shortcut_24', 'conv_25', 'bn_25', 'leaky_26', 'conv_26', 'bn_26', 'leaky_27', 'shortcut_27', 'conv_28', 'bn_28', 'leaky_29', 'conv_29', 'bn_29', 'leaky_30', 'shortcut_30', 'conv_31', 'bn_31', 'leaky_32', 'conv_32', 'bn_32', 'leaky_33', 'shortcut_33', 'conv_34', 'bn_34', 'leaky_35', 'conv_35', 'bn_35', 'leaky_36', 'shortcut

In [ ]:
# 인공신경망 네트워크 레이어 중 출력 레이어의 인덱스를 가져온다.
output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]

print("net.getUnconnectedOutLayers() len :", len(net.getUnconnectedOutLayers())) # net.getUnconnectedOutLayers() len : 3
print(net.getUnconnectedOutLayers())                                             # [200 227 254]
print(output_layers)                                                             # ['yolo_82', 'yolo_94', 'yolo_106']

net.getUnconnectedOutLayers() len : 3
[200 227 254]
['yolo_82', 'yolo_94', 'yolo_106']


In [ ]:
# 검출(Detection) 할 원본 비디오, 출력 파일
file_name = 'gdrive/My Drive/CV/Object Detection_YOLO/video/street.mp4'
output_name = 'yolov3_street_video_output.mp4'  # Detection 된 output 비디오
elapsed_time = 0                                # 총 경과시간 초기화

In [ ]:
# 원본 비디오에서 video stream을 읽고, 프레임 단위로 객체 검출을 수행한다.

cap = cv2.VideoCapture(file_name)
writer = None
if not cap.isOpened:
    print('--(!)Error opening video capture')
    exit(0)
while True:
    ret, frame = cap.read()
    if frame is None:
        # close the video file pointers
        cap.release()
        # close the writer point
        writer.release()
        print('--(!) No captured frame -- Break!')
        print("elapsed time {:.3f} seconds".format(elapsed_time))
        break
    detectAndDisplay(frame)

6 car: 99.80
8 bus: 92.04
15 car: 85.26
16 car: 93.64
17 car: 98.69
19 person: 91.41
22 person: 79.44
24 car: 71.02
26 car: 93.01
28 handbag: 54.26
29 handbag: 52.69
=== A frame took 2.368 seconds
4 car: 99.86
6 bus: 90.03
11 car: 84.90
13 car: 95.70
14 car: 99.18
15 person: 95.11
18 person: 86.28
20 car: 67.57
22 car: 95.93
24 handbag: 58.64
=== A frame took 0.873 seconds
3 car: 99.58
4 bus: 86.50
8 car: 92.51
10 car: 97.41
11 car: 99.21
12 person: 86.89
15 person: 82.37
20 bus: 81.09
21 car: 62.38
23 car: 92.06
24 handbag: 61.19
=== A frame took 1.015 seconds
2 car: 99.70
5 bus: 84.70
7 car: 96.38
9 car: 97.75
11 car: 99.60
12 person: 85.65
15 person: 70.29
17 bus: 67.67
19 car: 59.86
21 car: 94.20
22 handbag: 60.92
=== A frame took 1.312 seconds
2 car: 99.75
5 bus: 88.23
7 car: 94.84
8 car: 96.77
10 car: 99.31
12 person: 91.90
13 person: 76.75
16 bus: 84.25
18 car: 94.74
19 handbag: 58.03
=== A frame took 1.356 seconds
1 car: 99.57
3 bus: 90.23
9 car: 94.97
10 car: 96.53
13 car: 99.

In [ ]:
# 실행 결과 'yolov3_street_video_output.mp4' 파일 확인

!ls

gdrive	sample_data  yolov3_street_video_output.mp4


In [ ]:
# 내 구글 드라이브로 파일 이동

!mv yolov3_street_video_output.mp4 gdrive/My\ Drive/CV/Object\ Detection_YOLO/video/
print('yolov3_street_video_output.mp4 move complete!!')

yolov3_street_video_output.mp4 move complete!!


## YOLOv3-tiny

In [ ]:
# Load YOLOv3-tiny
net = cv2.dnn.readNet("gdrive/My Drive/CV/Object Detection_YOLO/config/yolov3-tiny.weights",
                      "gdrive/My Drive/CV/Object Detection_YOLO/config/yolov3-tiny.cfg")

In [ ]:
# YOLO 인공신경망 계층 이름 출력
layer_names = net.getLayerNames()
print("layer_name len :", len(layer_names))  # layer_name len : 48
print("layer_name:", layer_names)            # layer_name: ('conv_0', 'bn_0', 'leaky_1', 'pool_1', 'conv_2', 'bn_2',

layer_name len : 48
layer_name: ('conv_0', 'bn_0', 'leaky_1', 'pool_1', 'conv_2', 'bn_2', 'leaky_3', 'pool_3', 'conv_4', 'bn_4', 'leaky_5', 'pool_5', 'conv_6', 'bn_6', 'leaky_7', 'pool_7', 'conv_8', 'bn_8', 'leaky_9', 'pool_9', 'conv_10', 'bn_10', 'leaky_11', 'pool_11', 'conv_12', 'bn_12', 'leaky_13', 'conv_13', 'bn_13', 'leaky_14', 'conv_14', 'bn_14', 'leaky_15', 'conv_15', 'permute_16', 'yolo_16', 'identity_17', 'conv_18', 'bn_18', 'leaky_19', 'upsample_19', 'concat_20', 'conv_21', 'bn_21', 'leaky_22', 'conv_22', 'permute_23', 'yolo_23')


In [ ]:
# 인공신경망 네트워크 레이어 중 출력 레이어의 인덱스를 가져온다.
output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]

print("net.getUnconnectedOutLayers() len :", len(net.getUnconnectedOutLayers())) # layer_name: ('conv_0', 'bn_0', 'leaky_1', 'pool_1', 'conv_2', 'bn_2',
print(net.getUnconnectedOutLayers())                                             # [36 48]
print(output_layers)                                                             # ['yolo_16', 'yolo_23']

net.getUnconnectedOutLayers() len : 2
[36 48]
['yolo_16', 'yolo_23']


In [ ]:
# 검출(Detection) 할 원본 동영상
file_name = 'gdrive/My Drive/CV/Object Detection_YOLO/video/street.mp4'

output_name = 'yolov3tiny_street_video_output.mp4'  # Detection 된 output 동영상
elapsed_time = 0                                    # 총 경과시간 초기화

In [ ]:
# 원본 비디오에서 video stream을 읽고, 프레임 단위로 객체 검출을 수행한다.

cap = cv2.VideoCapture(file_name)
writer = None
if not cap.isOpened:
    print('--(!)Error opening video capture')
    exit(0)
while True:
    ret, frame = cap.read()
    if frame is None:
        # close the video file pointers
        cap.release()
        # close the writer point
        writer.release()
        print('--(!) No captured frame -- Break!')
        print("elapsed time {:.3f} seconds".format(elapsed_time))
        break
    detectAndDisplay(frame)

1 car: 90.96
3 car: 93.09
=== A frame took 0.500 seconds
1 car: 94.10
3 car: 88.68
=== A frame took 0.120 seconds
1 car: 96.99
2 car: 84.43
=== A frame took 0.127 seconds
1 car: 97.89
2 car: 95.68
=== A frame took 0.121 seconds
1 car: 96.45
2 car: 57.02
3 car: 96.06
=== A frame took 0.130 seconds
0 car: 98.78
1 car: 96.57
2 car: 63.08
=== A frame took 0.121 seconds
0 car: 96.69
2 car: 95.96
3 car: 82.62
=== A frame took 0.121 seconds
1 car: 99.01
2 car: 54.43
4 car: 86.15
5 car: 93.82
=== A frame took 0.126 seconds
0 car: 98.73
1 car: 61.01
3 car: 78.88
5 car: 92.20
=== A frame took 0.131 seconds
0 car: 97.57
2 car: 58.65
3 car: 73.85
5 car: 90.25
=== A frame took 0.117 seconds
0 car: 87.16
4 car: 83.24
5 car: 58.92
6 car: 89.31
=== A frame took 0.121 seconds
0 car: 85.95
2 car: 71.47
3 car: 68.79
4 car: 57.00
=== A frame took 0.145 seconds
0 car: 88.62
1 person: 63.97
2 car: 76.50
3 car: 72.56
4 car: 53.47
=== A frame took 0.143 seconds
1 car: 82.45
2 person: 53.52
3 car: 66.51
4 car:

In [ ]:
# 실행 결과 'yolov3tiny_street_video_output.mp4' 파일 확인

!ls

gdrive	sample_data  yolov3tiny_street_video_output.mp4


In [ ]:
# 내 구글 드라이브로 파일 이동

!mv yolov3tiny_street_video_output.mp4 gdrive/My\ Drive/CV/Object\ Detection_YOLO/video/
print('yolov3tiny_street_video_output.mp4 move complete!!')

yolov3tiny_street_video_output.mp4 move complete!!
